In [1]:
%load_ext autoreload
%autoreload 2
import os
import re
import json
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from scipy.io.wavfile import read, write
from pipefunc.pipefunc import do_command, send_command, get_response, get_response_json

pipe-test.py, running on windows
Write to  "\\.\pipe\ToSrvPipe"
Read from "\\.\pipe\FromSrvPipe"
-- Both pipes exist.  Good.
-- File to write to has been opened
-- File to read from has now been opened too



In [2]:
# create file list for input
p = Path(os.getcwd())
p_raw = p.parents[0].joinpath('raw')
m4afilelist = sorted(p_raw.glob('*.m4a'))
print(m4afilelist)

[WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP020/raw/Angel.m4a'), WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP020/raw/Liz.m4a'), WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP020/raw/Mike.m4a')]


In [6]:
def getinfo2json():
    send_command('GetInfo: Format="JSON" Type="Tracks"')
    response = get_response_json()
    result = json.loads(response)
    return(result)

trackinfo = getinfo2json()
print(trackinfo)

filename_noext = [re.sub('.wav', '', m4afile.name) for m4afile in m4afilelist]

namelistorder = ['Angel', 'Liz', 'Mike']
tracknamelist = [trackinfo['name'] for trackinfo in trackinfo]
name_tag = 'Liz'
audio_type = 'm4a'

def importfiles(filepath):
    do_command(f'Import2: Filename={filepath}')

TRACK_HEIGHT = 80
TRACK_OFFSET = 0
MATERIAL_DIR = Path('Z:\sc2.shih\Drive\Podcast\Edit\material')
ENDCREDIT_OFFSET = 9.2

for idx, name in enumerate(namelistorder):
    track_name = f'{name_tag}_{name}'
    if track_name not in tracknamelist:
        print(f'import track: {track_name}')
        importfiles(p_raw.joinpath(f'{track_name}.{audio_type}'))
    do_command('SelectAll:')        
    do_command(f'SelectTracks: Mode="Set" Track="{idx}" TrackCouint="1"')
    do_command('ZoomSel')
    do_command(f'SetTrack: Height={str(TRACK_HEIGHT)}')
    do_command(f'SetClip: At="0" Start="{TRACK_OFFSET}"')

trackinfo = getinfo2json()
tracknamelist = [trackinfo['name'] for trackinfo in getinfo2json()]
tracklengthlist = [trackinfo['end'] - trackinfo['start'] for trackinfo in trackinfo]
TRACK_LENGTH_MAX = np.amax(tracklengthlist)

Send: >>> 
GetInfo: Format="JSON" Type="Tracks"
[]
import track: Liz_Angel
Send: >>> 
Import2: Filename=c:\Users\michaelshih\Documents\Podcast_tmp\EP020\raw\Liz_Angel.m4a
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectAll:
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectTracks: Mode="Set" Track="0" TrackCouint="1"
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
ZoomSel
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SetTrack: Height=80
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SetClip: At="0" Start="0"
Rcvd: <<< 

BatchCommand finished: OK

import track: Liz_Liz
Send: >>> 
Import2: Filename=c:\Users\michaelshih\Documents\Podcast_tmp\EP020\raw\Liz_Liz.m4a
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectAll:
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectTracks: Mode="Set" Track="1" TrackCouint="1"
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
ZoomSel
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SetTrack: Height=80
Rcvd: <<< 

BatchC

In [5]:
import pandas as pd
from datetime import datetime, timedelta
df = pd.read_csv(f'addmusic.csv')

for index, row in df.iterrows():
    hhmmss = row['timestamp']
    filename = row['music']
    MIDDLEPATH = MATERIAL_DIR.joinpath('02_transition', filename)

    [hours, minutes, seconds] = [int(x) for x in hhmmss.split(':')]
    MIDDLE_TIMESTAMP = timedelta(hours=hours, minutes=minutes, seconds=seconds).seconds
    print(MIDDLE_TIMESTAMP)

    trackinfo = getinfo2json()
    tracknamelist = [trackinfo['name'] for trackinfo in getinfo2json()]

    clipname = f'middle_{index + 1}'
    if clipname not in tracknamelist:
        do_command(f'Import2: Filename="{MIDDLEPATH}"')
        do_command(f'SelectTracks: Mode="Set" Track="{len(tracknamelist)}" TrackCouint="1"')
        do_command(f'SetTrack: Name="{clipname}" Height={int(TRACK_HEIGHT*0.6)}')
        do_command(f'SetClip: At="0" Start="{TRACK_OFFSET + MIDDLE_TIMESTAMP}"')
    

948
Send: >>> 
GetInfo: Format="JSON" Type="Tracks"
Send: >>> 
GetInfo: Format="JSON" Type="Tracks"
Send: >>> 
Import2: Filename="Z:\sc2.shih\Drive\Podcast\Edit\material\02_transition\Middle_01.wav"
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectTracks: Mode="Set" Track="7" TrackCouint="1"
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SetTrack: Name="middle_1" Height=48
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SetClip: At="0" Start="977.6"
Rcvd: <<< 

BatchCommand finished: OK

1870
Send: >>> 
GetInfo: Format="JSON" Type="Tracks"
Send: >>> 
GetInfo: Format="JSON" Type="Tracks"
Send: >>> 
Import2: Filename="Z:\sc2.shih\Drive\Podcast\Edit\material\02_transition\Middle_01.wav"
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectTracks: Mode="Set" Track="8" TrackCouint="1"
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SetTrack: Name="middle_2" Height=48
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SetClip: At="0" Start="1899.6"
Rcvd: <<< 

BatchCommand fi

In [7]:
# compressor
trackinfo = getinfo2json()
tracknamelist = [trackinfo['name'] for trackinfo in getinfo2json()]

for idx, name in enumerate(namelistorder):
    track_name = f'{name_tag}_{name}'
    do_command('SelectAll:')        
    do_command(f'SelectTracks: Mode="Set" Track="{idx}" TrackCouint="1"')
    do_command('ZoomSel')
    do_command(f'Compressor: Threshold={str(-12)} NoiseFloor={str(-35)} Ratio={str(2)}')


Send: >>> 
GetInfo: Format="JSON" Type="Tracks"
Send: >>> 
GetInfo: Format="JSON" Type="Tracks"
Send: >>> 
SelectAll:
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectTracks: Mode="Set" Track="0" TrackCouint="1"
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
ZoomSel
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
Compressor: Threshold=-12 NoiseFloor=-35 Ratio=2
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectAll:
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectTracks: Mode="Set" Track="1" TrackCouint="1"
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
ZoomSel
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
Compressor: Threshold=-12 NoiseFloor=-35 Ratio=2
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectAll:
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
SelectTracks: Mode="Set" Track="2" TrackCouint="1"
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
ZoomSel
Rcvd: <<< 

BatchCommand finished: OK

Send: >>> 
Compressor: Threshold=-12 NoiseFloo